In [33]:
import pandas as pd
from unicodedata2 import normalize
import os
import glob
import re
import shutil

In [34]:
# Função para normalizar nomes de colunas, evitando erros futuros em salvamentos
def normalizar_colunas(dataframe):
    lista_colunas = []
    for coluna in dataframe.columns:
        nova_coluna = normalize('NFKD', coluna).encode('ascii', 'ignore').decode('utf-8').replace(' ', '_')
        lista_colunas.append(nova_coluna)
    dataframe.columns = lista_colunas
    return dataframe

# Função para remover partidas que não devam constar no arquivo final por estarem com o código errado
def remover_por_codigo():
    arquivos_excel = glob.glob(r"..\\Excel\\*.xlsx")
    for arquivo in arquivos_excel:
        nome_arquivo_excel = arquivo.split('\\')[-1]
        # Verifica se "REMOVER" está no nome do arquivo e se o código "TxxBxx" também está presente ("x" são números)
        if "REMOVER" in nome_arquivo_excel.upper() and re.search(r'T\d{2}B\d{2}', nome_arquivo_excel):
            codigo_partida = re.search(r'T\d{2}B\d{2}', nome_arquivo_excel).group(0)
            for arquivo_csv in glob.glob(r"..\\CSV\\01.bronze\\*.csv"):
                try:
                    nome_arquivo_csv = arquivo_csv.split('\\')[-1]
                    caminho_arquivo = f"..\\CSV\\01.bronze\\{nome_arquivo_csv}"
                    df = pd.read_csv(caminho_arquivo)
                
                    df_filtered = df[~df.apply(lambda row: row.astype(str).str.contains(codigo_partida).any(), axis=1)]
                    
                    df_filtered.to_csv(caminho_arquivo, index=False)
                    # os.remove(arquivo)
                except Exception as e:
                    if str(e).__contains__("Permission denied"):
                        print("Feche o arquivo antes de salvar!")
                    else:
                        print(e)

#TODO Alterar forma de salvamento para quando o valor encontrado já existir no arquivo bronze
def salvamento_csv(dataframe, nome_arquivo, nome_arquivo_excel):
    caminho_arquivo = f"..\\CSV\\01.bronze\\bronze_{nome_arquivo}.csv"
    if os.path.exists(caminho_arquivo):
        dataframe.to_csv(caminho_arquivo, mode='a', header=False, index=False)
        print(f"Anexando '{nome_arquivo_excel}' a 'bronze_{nome_arquivo}.csv'.")
    else:
        dataframe.to_csv(caminho_arquivo, mode='w', header=True, index=False)
        print(f"Criando 'bronze_{nome_arquivo}.csv' a partir de '{nome_arquivo_excel}'.")

In [46]:
remover_por_codigo()

caminho_excel = r"..\\Excel\\"
pasta_excel = glob.glob(f"{caminho_excel}\\*")
regex_arquivo_excel = r"Sumula_Teste_T[0-9][0-9]B[0-9][0-9].xlsx"
arquivos_excel = glob.glob(os.path.join(caminho_excel, regex_arquivo_excel))
for arquivo_excel in arquivos_excel:
    try:
        csv_indivudual = pd.read_csv(next(arquivo_excel for arquivo_excel in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "individual" in arquivo_excel.lower()))
        csv_times = pd.read_csv(next(arquivo_excel for arquivo_excel in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "times" in arquivo_excel.lower()))
        csv_info = pd.read_csv(next(arquivo_excel for arquivo_excel in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "info" in arquivo_excel.lower()))
        nome_arquivo_excel = arquivo_excel.split('\\')[-1]
        arquivo_completo = pd.read_excel(arquivo_excel, sheet_name=None, engine="calamine")
    except Exception as e:
                if str(e).__contains__("Permission denied"):
                    print("Feche o arquivo antes de salvar!")
                else:
                    print(e)

    df_individual = arquivo_completo["Individual"]
    df_times = arquivo_completo["Times"]
    df_info = arquivo_completo["Informações"]

    normalizar_colunas(df_individual)
    normalizar_colunas(df_times)
    normalizar_colunas(df_info)

    codigo_arquivo = re.search(r'T\d{2}B\d{2}', nome_arquivo_excel).group(0)
    codigo_baba_ind = df_individual["Baba"].unique()
    qtd_codigo_baba_ind = len(codigo_baba_ind)
    codigo_baba_times = df_times["Baba"].unique()
    qtd_codigo_baba_times = len(codigo_baba_times)
    codigo_validacao = df_info["Baba"][0]

    for valor in codigo_baba_ind:
        if valor != codigo_validacao:
            print(f"Há códigos do baba diferentes em '{nome_arquivo_excel}' na aba 'Individual'.")
        
    for valor in codigo_baba_times:
        if valor != codigo_validacao:
            print(f"Há códigos do baba diferentes em '{nome_arquivo_excel}' na aba 'Times'.")

    if (qtd_codigo_baba_ind == 1) and (qtd_codigo_baba_times == 1):
        if codigo_baba_ind == codigo_baba_times == codigo_validacao == codigo_arquivo:
            try:
                # print(codigo_arquivo)
                if codigo_arquivo in csv_info.iloc[:,0].unique():
                    # print(f"O código encontrado é {codigo_arquivo}")
                    csv_indivudual = csv_indivudual[~csv_indivudual.apply(lambda row: row.astype(str).str.contains(codigo_arquivo).any(), axis=1)]
                    csv_times = csv_times[~csv_times.apply(lambda row: row.astype(str).str.contains(codigo_arquivo).any(), axis=1)]
                    csv_info = csv_info[~csv_info.apply(lambda row: row.astype(str).str.contains(codigo_arquivo).any(), axis=1)]
                    csv_indivudual.to_csv(next(arquivo_excel for arquivo_excel in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "individual" in arquivo_excel.lower()), index=False)
                    csv_times.to_csv(next(arquivo_excel for arquivo_excel in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "times" in arquivo_excel.lower()), index=False)
                    csv_info.to_csv(next(arquivo_excel for arquivo_excel in glob.glob(r"..\\CSV\\01.bronze\\*.csv") if "info" in arquivo_excel.lower()), index=False)
                salvamento_csv(df_individual, "desempenho_individual", nome_arquivo_excel)
                salvamento_csv(df_times, "desempenho_times", nome_arquivo_excel)
                salvamento_csv(df_info, "info_partidas", nome_arquivo_excel)
                destino_historico = r"..\\Excel_Historico\\"
                os.makedirs(destino_historico, exist_ok=True)
                shutil.move(arquivo_excel, destino_historico)
            except Exception as e:
                if str(e).__contains__("Permission denied"):
                    print("Feche o arquivo antes de salvar!")
                else:
                    print(e)
        else:
            print(f"Há códigos de baba diferentes em '{nome_arquivo_excel}'.")
    else:
            print(f"Há códigos de baba diferentes em '{nome_arquivo_excel}'.")
print("------------------------------------------------------------------")
# Atualização da variável "pasta_excel" para pegar somente os caminhos restantes, ignorando os iniciais
pasta_excel = glob.glob(f"{caminho_excel}\\*")
for outro_arquivo_excel in pasta_excel:
    nome_outro_arquivo_excel = outro_arquivo_excel.split('\\')[-1]
    print(f"O arquivo com nome '{nome_outro_arquivo_excel}' não pôde ser processado.")

Anexando 'Sumula_Teste_T01B01.xlsx' a 'bronze_desempenho_individual.csv'.
Anexando 'Sumula_Teste_T01B01.xlsx' a 'bronze_desempenho_times.csv'.
Anexando 'Sumula_Teste_T01B01.xlsx' a 'bronze_info_partidas.csv'.
Anexando 'Sumula_Teste_T01B02.xlsx' a 'bronze_desempenho_individual.csv'.
Anexando 'Sumula_Teste_T01B02.xlsx' a 'bronze_desempenho_times.csv'.
Anexando 'Sumula_Teste_T01B02.xlsx' a 'bronze_info_partidas.csv'.
Há códigos do baba diferentes em 'Sumula_Teste_T01B03.xlsx' na aba 'Individual'.
Há códigos do baba diferentes em 'Sumula_Teste_T01B03.xlsx' na aba 'Times'.
Há códigos de baba diferentes em 'Sumula_Teste_T01B03.xlsx'.
Anexando 'Sumula_Teste_T01B04.xlsx' a 'bronze_desempenho_individual.csv'.
Anexando 'Sumula_Teste_T01B04.xlsx' a 'bronze_desempenho_times.csv'.
Anexando 'Sumula_Teste_T01B04.xlsx' a 'bronze_info_partidas.csv'.
------------------------------------------------------------------
O arquivo com nome 'REMOVER_T01B02.xlsx' não pôde ser processado.
O arquivo com nome 'Su